In [2]:

##################### ARDUINOLIB STARTS HERE ###########################
from serial import Serial

class Arduino:
  """
  A wrapper class around a serial port with some helpers for easy communication to/from an Arduino.

  Example:

  ```python
  with Arduino('/dev/com.whatever', baudRate=115200) as arduino:
    for line in arduino.lines():
      print("Got line from Arduino: " + line)
      arduino.write(55) # Writes 0x55 as a byte. Also accepts raw bytes or strings.
  ```
  """

  def __init__(self, port, baudRate=115200, timeout=1, logging=False):
    """
    Connect to an Arduino
    """
    self.logging = logging
    self.serial_port = Serial(port, baudRate, timeout=timeout)

  def _log(self, *args, **kwargs):
    """
    Helper method that acts like `print`, when logging=True but does nothing otherwise.
    """
    if self.logging:
      print("[Arduino]", *args, **kwargs)
  
  # We want to just pass through to the serial port's context manager
  def __enter__(self):
    """
    When using an Arduino as a context manager, the Arduino will intelligently open/close the serial
    port upon entering/exiting the context manager, including doing so multiple times.
    """
    self._log("Entering Arduino context manager, connecting serial port...")
    self.serial_port.__enter__()

    # But return self so you can do `with Arduino(...) as arduino:`
    return self

  def __exit__(self, __exc_type, __exc_value, __traceback):
    """
    When using an Arduino as a context manager, the Arduino will intelligently open/close the serial
    port upon entering/exiting the context manager, including doing so multiple times.
    """
    self._log("Exiting Arduino context manager, disconnecting serial port...")
    return self.serial_port.__exit__(__exc_type, __exc_value, __traceback)
  
  # NB: Calling lines() or packets() more than once is undefined behavior
  def lines(self, drain_first=True):
    """
    Return an iterator that yields each line the Arduino sends over the Serial connection.

    If drain_first is True, any serial data already received and buffered but not yet processed will
    be erased.

    NOTE: This iterator will block while waiting for a line
    NOTE: Calling this method more than once, or calling it after packets() has been called, is
          undefined behavior.
    """
    if drain_first:
      self.serial_port.reset_input_buffer()

    while True:
      # NOTE: technically this would get rid of leading spaces too if that was something you cared about
      line = self.serial_port.readline().decode('ascii').strip()
      if len(line) > 0:
        self._log(f"Received Line: {line}")
        yield line

  def packets(self, drain_first=True):
    """
    Return an iterator that yields each packet the Arduino sends over the Serial connection.

    A packet is defined as a newline-terminated, comma-separated list of integers. In other words,
    this method expects that your Arduino writes data over serial that looks like this: `1,2,3\n`.

    If drain_first is True, any serial data already received and buffered but not yet processed will
    be erased.

    NOTE: This iterator will block while waiting for a line
    NOTE: Calling this method more than once, or calling it after lines() has been called, is
          undefined behavior.
    """
    for line in self.lines(drain_first=drain_first):
      packet = tuple(int(data) for data in line.split(','))
      self._log(f"Received Packet: {packet}")
      yield packet

  def write(self, data):
    """
    Write data to the Arduino over Serial. If data is bytes, it will be sent as-is. If data is an
    int, it will be converted to an unsigned 8-bit integer and sent that way (attempting to write an
    integer outside of the range 0-255 is an error). If data is a string it will be utf-8 encoded.
    If data is a list each element will be individually written as per the above rules.
    """
    self._log(f"Writing data (may need conversion to bytes): {data}")

    if not isinstance(data, bytes):
      if isinstance(data, str):
        self.write(data.encode('ascii'))
      elif isinstance(data, int):
        self.write(data.to_bytes(1, 'big', signed=True))
      elif isinstance(data, list):
        for data_item in data:
          self.write(data_item)
      else:
        raise Exception("Cannot write data of unknown type!")
    else:
      for byte in data:
        self.serial_port.write(byte)
  
  def writeln(self, data):
    """
    Write a string to the Arduino over Serial, and add a newline at the end.
    """
    return self.write(f"{data}\n")
##################### ARDUINOLIB ENDS HERE ###########################


In [3]:
def pairs(iterable):
	iterator = iter(iterable)
	try:
		while True:
			yield next(iterator), next(iterator)
	except StopIteration:
		pass

def multinext(iterator, n):
	items = []
	while len(items) < n:
		items.append(next(iterator))
	return tuple(items)

In [4]:
class Config:
	def __init__(self, on=100, off=1000):
		self.on = on
		self.off = off

	def send(self, arduino):
		arduino.serial_port.write((0 << 8) + self.on)
		arduino.serial_port.write((1 << 8) + self.off)
	
	def __repr__(self):
		return f"Config(on={self.on}, off={self.off})"

In [20]:
with Arduino('/dev/cu.usbmodem14301', logging=True) as arduino:
	conf_mostlyoff = Config(on=5, off=6)

	read = lambda: arduino.serial_port.read(size=1)

	params = [-1] * 8

	while True:
		align_state = 0
		while True:
			data = read()
			if len(data) == 0: continue
			if align_state in [0, 2]:
				if data == b'\xFF':
					align_state += 1
				else:
					align_state = 0
					continue
			elif align_state in [1, 3]:
				if data == b'\xAA':
					align_state += 1
					if align_state == 4:
						break
				else:
					align_state = 0
					continue

		alldata = []

		while len(alldata) < 16:
			data = read()
			if len(data) == 0: continue
			alldata.append(data)
	
		for idx, value in pairs(alldata):
			idx_int = list(idx)[0]
			val_int = list(value)[0]
			if idx_int >= len(params):
				print(f"ERROR: Index out of bounds: {idx_int}. Realigning...")
				continue
			if val_int != params[idx_int]:
				params[idx_int] = val_int
				print(f"{idx_int:02} -> {val_int:02X}")
			else:
				if idx_int == 7:
					# arduino.serial_port.write(bytearray([0, 123]))
					arduino.serial_port.write(bytearray([1]))
					arduino.serial_port.write(bytearray([5]))




	

[Arduino] Entering Arduino context manager, connecting serial port...
00 -> 00
01 -> 00
02 -> 00
03 -> 00
04 -> 00
05 -> 00
06 -> 00
07 -> 00
01 -> 05


In [ ]:
with Arduino('/dev/cu.usbmodem142201') as arduino:
	conf_mostlyon = Config(on=1000, off=100)
	conf_mostlyon.send(ardunio)